<a href="https://colab.research.google.com/github/anadiedrichs/rl-ai/blob/master/1_FrozenLake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio #1 de Aprendizaje por refuerzo - Inteligencia Artificial - DISI 2019

**Realizado por: Ing. Ana Laura Diedrichs**

Consultas: lunes 19 hs en sala consulta de sistemas.

Contacto por otros horarios de consulta o dudas: 
* Email: ana.diedrichs@frm.utn.edu.ar
* Telegram  @anadiedrichs
* en el grupo de la materia por dudas generales

## Objetivos del laboratorio:

* Mediante un problema de juguete evaluar las diferencias entre el comportamiento aleatorio de un agente y su comportamiento con aprendizaje por refuerzo
* Comprensión del funcionamiento del algoritmo de iteración de valores
* Aplicar aprendizaje por refuerzo a un entorno sencillo.
* Ser una entrada en calor o introducción del uso del entorno colab.research.google.com y la librería Open AI Gym
* Fomentar la aplicación de conceptos teóricos aprendidos al laboratorio.

## Pre-requisitos o pre-condiciones
* Tener una cuenta google (gmail)
* Tener instalado el navegador google chrome 
* Contar con conectividad a Internet 


## Entrega y uso del laboratorio

**USO**

* Antes que cualquier cosa, **cree una copia de este notebook: Click en *File*, luego *Save a Copy in Drive***
* Renombre el archivo con el siguiente formato: APELLIDO_NOMBRE_LEGAJO_titulonotebook.ipynb 
Ejemplo: DIEDRICHS_ANA_99999_FrozenLake.ipynb
* Use el notebook, complete las actividades y consignas que se elija. 
* Este laboratorio es una actividad individual.
* Se fomenta el intercambio de opiniones en clase.

**ENTREGA**

* Una vez finalizado el laboratorio, complete [el formulario de entrega](https://forms.gle/AqdeVPA38chsJqR99) indicando
 * Apellido
 * Nombre
 * Nro Legajo
 * Turno (tarde o noche)
 * link de su notebook. El mismo se obtiene si realiza click en *Share* (esquina superior derecha) y luego en *Get shareable link* 
 
 
 No se aceptarán otras formas de entrega distintas a la mencionada.
 
 Fecha límite de entrega: se indicará en clase o por la lista de correos. Última semana de clases es la entrega.

## Instalando y cargando librerías

Instamos y cargamos librerias que vamos a usar en este ejemplo

In [0]:
!pip install cmake 'gym' scipy

In [0]:
import numpy as np
import gym
import pandas as pd
import matplotlib.pyplot as plt
import gym.spaces as spaces
import time
from time import sleep

## Problema FrozenLake

Trabajaremos con el ambiente FrozenLake o lago congelado. El objetivo del agente es ir de un origen a un destino sin caer en agujeros.

In [0]:
env = gym.make('FrozenLake-v0') # Trabajaremos con el ambiente FrozenLake
env.seed(1234) # semilla para generador random
env.render() # nos muestra una imagen de como luce el ambiente.

### ACTIVIDAD 

Leyendo [la documentación](https://gym.openai.com/envs/FrozenLake-v0/) del ambiente FrozenLake

 

#### 1.a) ESTADOS: Explique el ambiente, qué significan la F, H G y S

**SU RESPUESTA AQUI**

#### 1.b) ACCIONES: El siguiente código nos muestra el espacio de acciones del agente. ¿Cuáles son las cuatro acciones que puede realizar el agente?

In [0]:
env.action_space

**SU RESPUESTA AQUI**

#### 1.c) ¿Cuáles son los valores de recompensa/penalización y cuándo la recibe? Explique

**SU RESPUESTA AQUI**

## Tabla de probabilidades de transición estado - acción

![Imagen dibujo de los estados](https://twice22.github.io/images/rl_series/frozenlake.png)

El atributo env.P del objeto *environment* de Open AI Gym guarda la tabla de transición de probabilidades estado-acción, o función de transición.

In [0]:
env.env.P

Para el estado 0, vemos las probabilidades de transición

In [0]:
env.env.P[14]

Si el estado es 0 y la acción es 2 (ir a la derecha)

In [0]:
env.env.P[0][2]

El primer número, 0.33 es la probabilidad de transición.

El segundo, 4, es el siguiente estado, 

El tercero la recompensa, 0 en este caso.

El cuarto ítem, False, es si finaliza el episodio.

Note que estando en el estado 0 y ejecutando la acción 2 el agente tiene iguales probabilidades de permanecer en el mismo estado (el cero), pasar al estado 1 o pasar al estado 4

### Actividad ¿qué interpreta de la siguiente línea de código? Indique desde qué estado se parte, la acción que se ejecuta y qué significa la salida.

In [0]:
env.env.P[6][2]

Si estoy en el estado 15 y ejecuto la acción 2

In [0]:
env.env.P[14][2]

### Actividad: ¿qué observa de los valores de probabilidad de la tabla env.env.P o $T(s,a,s')$ ? ¿qué podemos decir del agente/entorno?


## Agente aleatorio

Creamos una clase, RandomAgent, que sería un agente aleatorio, es decir, la siguiente acción a tomar la elige al azar.

In [0]:
class RandomAgent():
	def __init__(self,action_space):
		self.action_space = action_space
	
	def elegir_accion(self,observation):
		## Regresa una acción aleatoria 
		return self.action_space.sample()

Observe y ejecute la siguiente línea varias veces. ¿Qué regresa elegir_accion?


In [0]:
agente = RandomAgent(env.action_space)
obs=env.reset()
env.seed(1234) 
agente.elegir_accion(obs)

## Experimento con el agente aleatorio

Función que define si cayó en un hoyo o no.
Podría modificarla si lo desea o fuera necesario. 
Interprete su funcionamiento.

In [0]:
def es_un_pozo(obs,reward,done):
  es_pozo = False
  if(reward <= 0 and done == True):
    es_pozo = True
  
  return es_pozo
  

In [0]:
agent = RandomAgent(env.action_space)

all_rewards=[] # guardamos recompensas
frames=[] # para guardar los frames y luego mostrar el comportamiento del agente
contador=0 # contador de acciones o pasos datos en total
caidas_al_lago=0

for _ in range(1000):

  obs=env.reset()
  total_reward = 0
	
  while True:
    
    action = agent.elegir_accion(obs)
    obs,reward,done,info = env.step(action) # la acción se ejecuta en el ambiente
    contador+=1
    
    if(es_un_pozo(obs,reward,done)): 
      caidas_al_lago+=1
    
    # para renderizar
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': obs,
        'action': action,
        'reward': reward 
    })
    
    if done: # si terminó el episodio (llega al objetivo o cae en un hoyo)
      all_rewards.append(reward) # guarda la recompensa
      break


Ejecute el siguiente bloque de código para visualizar gráficamente el comportamiento del agente

In [0]:
len(frames) # cantidad total de frames, o acciones a mostrar del agente

In [0]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'].getvalue())
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
# descomente la siguiente línea        
#print_frames(frames) #si desea ver todos los frames, puede demorar bastante

In [0]:
#print_frames(frames)

Recompensa promedio de agente aleatorio.



In [0]:
len(all_rewards)


In [0]:
print ("Recompensa promedio: ", np.mean(all_rewards))

#### 1.d) Cuántas veces llegó al objetivo

In [0]:
# calcule lo que responde a la pregunta

Evaluacion de la recompensa en el tiempo de la recompensa

In [0]:
df = pd.DataFrame(all_rewards)
# variacion de los valores por cada iteracion
plt.plot(df)


Recompensa acumulada

In [0]:
# recompensa acumulada
plt.plot(df.cumsum())

## Iteración de valores

Creamos una clase que represente un agente que ejecuta iteración de valores, es la clase ValueIterAgent

![alt text](https://github.com/waqasqammar/MDP-with-Value-Iteration-and-Policy-Iteration/raw/d66dfe7aad0d23d7956aa87f05989a66a15202f6/nb_images/value_iter.png)

![alt text](https://www.dropbox.com/s/ix76ita6h3igdde/iteracion-de-valores.jpg?dl=1)

In [0]:
def one_step_lookahead(env, state, V , discount_factor = 0.99):
    """
    Helper function to  calculate state-value function
    
    Arguments:
        env: openAI GYM Enviorment object
        state: state to consider
        V: Estimated Value for each state. Vector of length nS
        discount_factor: MDP discount factor
        
    Return:
        action_values: Expected value of each action in a state. Vector of length nA
    """
    
    # initialize vector of action values
    action_values = np.zeros(env.nA)
    
    # loop over the actions we can take in an enviorment 
    for action in range(env.nA):
        # loop over the P_sa distribution.
        for probablity, next_state, reward, info in env.P[state][action]:
             #if we are in state s and take action a. then sum over all the possible states we can land into.
            action_values[action] += probablity * (reward + (discount_factor * V[next_state]))
            
    return action_values

In [0]:
import sys
class ValueIterAgent(): # clase ValueIterAgent
  #función constructora
	def __init__(self,env,gamma):
		self.max_iterations = 1000
		self.gamma = gamma
		self.num_estados=env.observation_space.n
		self.num_actions=env.action_space.n
		self.state_prob = env.env.P # tabla T(s,a,s')
		self.values = np.zeros(env.observation_space.n) # valores de utilidad de los estados
		self.policy = np.zeros(env.observation_space.n) # política
    
  #función de iteración de valores
	def value_iteration(self):
	    for i in range(self.max_iterations): # hasta un nro máximo de iteraciones
	        prev_v = np.copy(self.values)
	        for state in range(self.num_estados): # por cada estado
	            Q_value = []
	            for action in range(self.num_actions): # por cada acción 
	                next_states_rewards = []
                  # dado un estado y una acción, por cada tupla:
                  # P(transicion de estado) + próximo estado + recompensa
	                for trans_prob, next_state, reward_prob, _ in self.state_prob[state][action]: 
	                   
	                    next_states_rewards.append((trans_prob * (reward_prob + self.gamma * prev_v[next_state]))) 
                      
	                
	                Q_value.append(sum(next_states_rewards)) # suma de todas las recompensas futuras
                  
	            self.values[state] = max(Q_value) # asignar la mayor utilidad futura para ese estado
	            #print(state,Q_value)
	            #sleep(5)
	    return self.values
  
  #función de extraer la política
	def extract_policy(self):
	   
	    for s in range(self.num_estados):
	        q_sa = np.zeros(self.num_actions)
	        for a in range(self.num_actions):
	            for prox_estado in self.state_prob[s][a]:
                
	                # next_sr es una tupla de (probability, next state, reward, done)
	                p, s_, r, _ = prox_estado
                  
	                q_sa[a] += (p * (r + self.gamma * self.values[s_]))
                  
	        self.policy[s] = np.argmax(q_sa)   
          
  # regresa una acción dada una observación del ambiente, según la política
	def choose_action(self,observation):
			
		return self.policy[observation]  	


In [0]:
env.env.P[14]

## Agente con iteración de valores

In [0]:
gamma = 1

agent = ValueIterAgent(env,gamma)

agent.value_iteration();
agent.extract_policy();

print( "La política del agente: ", agent.policy)
all_rewards=[]
frames=[]



In [0]:
print("Los valores de Utilidades son \n",agent.values)

Como es un arreglo, lo formateo a una dimension 4x4

In [0]:
agent.values.reshape(4,4)

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(4, 4))
sns.heatmap(agent.values.reshape(4,4),  cmap="YlGnBu", annot=True, cbar=False);

In [0]:
gamma = 1

agent = ValueIterAgent(env,gamma)

agent.value_iteration();
agent.extract_policy();

print( "La política del agente: ", agent.policy)

### Actividad 2



In [0]:
agent.extract_policy();

print( "La política del agente: ", agent.policy)

#### 2.a) Dibuje  la política que toma el agente sobre los estados posibles

Ejemplo de un dibujito de los estados y políticas

|  |    | ||
|------|------|------|------|
|S derecha |F abajo |F izquierda |F abajo|
|F derecha |H abajo |F izquierda |H izquierda|
|F derecha |F abajo |F izquierda |H izquierda|
|H derecha |F abajo |F izquierda |G arriba |

Otro ejemplo (queda a su imaginación los símbolos o representación :-) )


|  |    | ||
|------|------|------|------|
|S <- |F v |F <- |F v|
|F &#8594; |H &#8595; |F &#8592; |H &#8592;|
|F derecha |F abajo |F izquierda |H izquierda|
|H derecha |F &#8595; |F izquierda |G &#8593;  |


Ud. debe interpretar el resultado de la salida de ejecutar la línea 

```
print( "La política del agente: ", agent.policy)
```

#### 2.b) Actividad

¿Qué interpreta de la política obtenida? (cuadro anterior) Cuente lo que ve, detalle y explique con su intuición

**su respuesta**

### Actividad 3 experimento iteración de valores

Corra el siguiente script.

In [0]:
gamma = 1

agent = ValueIterAgent(env,gamma)

agent.value_iteration();

agent.extract_policy();

print( "La política del agente: ", agent.policy)
all_rewards=[]
frames=[]

caidas_al_lago=0

for _ in range(1000):

  obs=env.reset()
  total_reward = 0
	
  while True:

    action = agent.choose_action(obs)
    obs,reward,done,info = env.step(action)

    if(es_un_pozo(obs,reward,done)): 
      caidas_al_lago+=1

    # descomente la siguiente línea si desea guardarl os frames para luego visualizar el comportamiento del agente
    
    #frames.append({'frame': env.render(mode='ansi'),'state': obs,'action': action,'reward': reward })

    if done:
      all_rewards.append(reward)
      break

Añada y ejecute los bloques de código y bloques de texto para responder:

* Cuántas veces llegó al objetivo
* Recompensa promedio del experimento
* Gráfica der recompensa acumulada
* Cuántas veces se cayó al lago

### Actividad 4, medir tiempo de ejecución

El siguiente script es un ejemplo de como medir el tiempo de duración de un proceso.

In [0]:
tic = time.time() # estampa inicial de tiempo. 
# llame a las funciones que desea ejecutar
sleep(5) # la función sleep el valor es en segundos
toc = time.time() # estampa final de tiempo
# calculo la diferencia entre valor final e inicio de tiempo
duración_total = (toc - tic) * 1000 # *1000 para ver el tiempo en milisegundos
duración_total

Agrege a los scripts anteriores: agente aleatorio y agente por iteración de valores las mediciones de tiempo total de ejecución.

### Actividad 5 

Modifique el valor de gamma del experimento de iteración por valores a 0.7 ( u otro valor menor a 1). 

¿Cuál es el valor de recompensa promedio obtenida?

¿Es mejor o peor el compotamiento del agente respecto al experimento con gamma = 1 ? 

¿Por qué lo cree? Justifique



In [0]:
# copie y pegue código para reproducir el experimento de iteración de valores con gamma a 0.7

## Iteración de políticas

![alt text](https://www.dropbox.com/s/gi0nnc1pgq2ug28/iteracion-de-politicas-.jpg?dl=1)



![alt text](https://github.com/waqasqammar/MDP-with-Value-Iteration-and-Policy-Iteration/raw/d66dfe7aad0d23d7956aa87f05989a66a15202f6/nb_images/policy_iter.png)

In [0]:
def one_step_lookahead(env, state, V , discount_factor = 0.99):
    """
    Función ayuda o auxiliar para calcular estado-valor 
    
    Arguments:
        env: objeto del ambiente openAI GYM 
        state: estado a considerar
        V: valor estimado para cada estado. Vector de longitud nS.
        discount_factor: factor de descuento MDP.
        
    Return:
        action_values: Valor esperado por cada acción en un estado. Vector de longitud nA
    """
    
    # inicitaliza el vector de valores d acciones
    action_values = np.zeros(env.nA)
    
    # itera sobre las acciones que podemos tomar del ambiente
    for action in range(env.nA):
        # itera sobre la distribución P_sa (probabilidad estado/acción)
        for probablity, next_state, reward, info in env.P[state][action]:
             #if we are in state s and take action a. then sum over all the possible states we can land into.
            action_values[action] += probablity * (reward + (discount_factor * V[next_state]))
            
    return action_values

In [0]:
def update_policy(env, policy, V, discount_factor):
    
    """
    Helper function to update a given policy based on given value function.
      
    
    Arguments:
        env: objeto del ambiente openAI GYM 
        state: estado a considerar
        V: valor estimado para cada estado. Vector de longitud nS.
        discount_factor: factor de descuento MDP.
    Return:
        policy: Política actualizada basada en la función estado-Valor o estado-Utilidad 'V'.
    """
    
    for state in range(env.nS):
        # Dado un estado, computar el valor estado-acción.
        action_values = one_step_lookahead(env, state, V, discount_factor)
        
        # elige una acción que maximice el valor de utilidad de estado-acción.
        policy[state] =  np.argmax(action_values)
        
    return policy

In [0]:
def value_iteration(env, discount_factor = 0.999, max_iteration = 1000):
    """
    Algorithm to solve MPD.
    
    Arguments:
        env: openAI GYM Enviorment object.
        discount_factor: MDP discount factor.
        max_iteration: Maximum No.  of iterations to run.
        
    Return:
        V: Optimal state-Value function. Vector of lenth nS.
        optimal_policy: Optimal policy. Vector of length nS.
    
    """
    # intialize value fucntion
    V = np.zeros(env.nS)
    
    # iterate over max_iterations
    for i in range(max_iteration):
        
        #  keep track of change with previous value function
        prev_v = np.copy(V) 
    
        # loop over all states
        for state in range(env.nS):
            
            # Asynchronously update the state-action value
            #action_values = one_step_lookahead(env, state, V, discount_factor)
            
            # Synchronously update the state-action value
            action_values = one_step_lookahead(env, state, prev_v, discount_factor)
            
            # select best action to perform based on highest state-action value
            best_action_value = np.max(action_values)
            
            # update the current state-value fucntion
            V[state] =  best_action_value
            
        # if policy not changed over 10 iterations it converged.
        if i % 10 == 0:
            # if values of 'V' not changing after one iteration
            if (np.all(np.isclose(V, prev_v))):
                print('Value converged at iteration %d' %(i+1))
                break

    # intialize optimal policy
    optimal_policy = np.zeros(env.nS, dtype = 'int8')
    
    # update the optimal polciy according to optimal value function 'V'
    optimal_policy = update_policy(env, optimal_policy, V, discount_factor)
    
    return V, optimal_policy


In [0]:
env = gym.make('FrozenLake-v0')
tic = time.time()
opt_V, opt_Policy = value_iteration(env.env, max_iteration = 1000)
toc = time.time()
elapsed_time = (toc - tic) * 1000
print (f"Time to converge: {elapsed_time: 0.3} ms")
print('Optimal Value function: ')
print(opt_V.reshape((4, 4)))
print('Final Policy: ')
print(opt_Policy)



In [0]:
def policy_eval(env, policy, V, discount_factor):
    """
    Helper function to evaluate a policy.
    
    Arguments:
        env: openAI GYM Enviorment object.
        policy: policy to evaluate.
        V: Estimated Value for each state. Vector of length nS.
        discount_factor: MDP discount factor.
    Return:
        policy_value: Estimated value of each state following a given policy and state-value 'V'. 
        
    """
    policy_value = np.zeros(env.nS)
    for state, action in enumerate(policy):
        for probablity, next_state, reward, info in env.P[state][action]:
            policy_value[state] += probablity * (reward + (discount_factor * V[next_state]))
            
    return policy_value

In [0]:
def policy_iteration(env, discount_factor = 0.999, max_iteration = 1000):
    """
    Algorithm to solve MPD.
    
    Arguments:
        env: openAI GYM Enviorment object.
        discount_factor: MDP discount factor.
        max_iteration: Maximum No.  of iterations to run.
        
    Return:
        V: Optimal state-Value function. Vector of lenth nS.
        new_policy: Optimal policy. Vector of length nS.
    
    """
    # intialize the state-Value function
    V = np.zeros(env.nS)
    
    # intialize a random policy
    policy = np.random.randint(0, 4, env.nS)
    policy_prev = np.copy(policy)
    
    for i in range(max_iteration):
        
        # evaluate given policy
        V = policy_eval(env, policy, V, discount_factor)
        
        # improve policy
        policy = update_policy(env, policy, V, discount_factor)
        
        # if policy not changed over 10 iterations it converged.
        if i % 10 == 0:
            if (np.all(np.equal(policy, policy_prev))):
                print('policy converged at iteration %d' %(i+1))
                break
            policy_prev = np.copy(policy)
            

            
    return V, policy

In [0]:

enviorment2 = gym.make('FrozenLake-v0')
tic = time.time()
opt_V2, opt_policy2 = policy_iteration(enviorment2.env, discount_factor = 0.999, max_iteration = 10000)
toc = time.time()
elapsed_time = (toc - tic) * 1000
print (f"Tiempo para converger: {elapsed_time: 0.3} ms")
print('Función de valores/utilidades óptima: ')
print(opt_V2.reshape((4, 4)))
print('Política final: ')
print(opt_policy2)

### Actividad 4

Escriba sus conclusiones y comparaciones entre el experimento del agente con comportamiento aleatorio, el de iteración por valores y el de iteración de políticas


**SU RESPUESTA AQUI**

### ACTIVIDAD 5 - Agrandando el ambiente 

En los ejercicios anteriores el ambiente tenía 16 estados, un espacio de 4x4 celdas donde se movía el agente. Ahora pasamos a un espacio de 8x8

In [0]:
env_name  = 'FrozenLake8x8-v0'
gamma = 1.0
env = gym.make(env_name)
env.render()

Ejecute para este nuevo ambiente un agente aleatorio, otro por iteración de políticas y otro por iteración de valores. 

Copie y pegue, reutilice código anterior. Añada todos los bloques de código que crea necesario

#### ACTIVIDAD 5.1 Conclusiones

Escriba conclusiones generales sobre los experimentos realizados en la actividad anterior.

Compare. ¿Qué cambios evidencia de pasar de un espacio de estados 4x4 a 8x8? Explique, detalle, conteste lo más completo posible. 

## Referencias


* Reinforcement Learning with OpenAI Gym - Value Iteration Frozen Lake - Code Heroku - [enlace](https://www.slideshare.net/codeheroku/reinforcement-learning-with-openai-gym-value-iteration-frozen-lake-code-heroku)
*  Frozen Lake environment in Open AI Gym [enlace a código del repo](https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py)
* Open AI Gym (https://gym.openai.com/)
* Iteración de valores, blog, por Holly Grimm  https://hollygrimm.com/rl_mdp
* Victor Busa, Open AI Gym intro, https://twice22.github.io/rl-part1/